# Imports

In [1]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from snowballstemmer.dutch_stemmer import DutchStemmer
from nltk.corpus import stopwords

In [7]:
import re

# Data Reading

In [8]:
data = pd.read_excel('Heijmans data final.xlsx')

In [9]:
data = data[['Categorie ongeval','Beschrijving incident']].dropna()

In [9]:
print(data)

                        Categorie ongeval  \
0                              Aanrijding   
1                              Aanrijding   
2                              Aanrijding   
3                              Aanrijding   
4                              Aanrijding   
...                                   ...   
3302  Vallen (van personen of voorwerpen)   
3303  Vallen (van personen of voorwerpen)   
3304  Vallen (van personen of voorwerpen)   
3305  Vallen (van personen of voorwerpen)   
3306  Vallen (van personen of voorwerpen)   

                                  Beschrijving incident  
0     Materialen in de bus slingeren los. Imperiaal ...  
1     Slachtoffer is aangereden in zijn bedrijfsauto...  
2     Fietspad niet afgeschermd van werkverkeer/terr...  
3     Aanrijding van een verkeersregelaar door een p...  
4     Veegwagen van Wolfswinkel reed (rond 14:50)teg...  
...                                                 ...  
3302  Bij het legen / openen van een big bag is Henk.

In [10]:
data.columns = ['Category', 'Text']

In [11]:
cats = ['Aanrijding',
'Agressie en extreme lichamelijke inspanning',
'Andere vormen van contact met voorwerpen',
'BHV inzet op locatie',
'Gevaarlijke stoffen, brand en explosies',
'Machines en handgereedschap',
'Vallen (van personen of voorwerpen)']


In [12]:
data = data[data['Category'].isin(cats)]

# Data Cleaning

### The following are done to clean the data:
- Lowering all the text
- Removing dutch stopwords
- Stemming

In [13]:
dutch = DutchStemmer()

In [14]:
all_stopwords = stopwords.words('dutch')

In [16]:
print(all_stopwords)

['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'niet', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere']


In [15]:
def clean_text(text):
    text = text.lower() # Convert the text to lower case
    text = re.sub(',', ' ', text) # Replace commas with an extra space
    
    text = re.sub('<.*?>', '', text) # Clean out any HTML tags
    text = re.sub('\s+',' ', text) # Replace multiple spaces with 
    
    text = text.split()
    
    text = [re.sub('[^\w]', '', i.rstrip()) for i in text if i not in all_stopwords] # Clean out stopwords
    
    text = dutch.stemWords(text)# Dutch Stemming
    
    text = ' '.join(text)
    return text

In [16]:
data['Text'] = data['Text'].apply(clean_text)
data['Text'].describe()

count                                     2826
unique                                    2521
top       ligg tevel losligg material bouwplat
freq                                        27
Name: Text, dtype: object

# Model without up/down sampling 

In [17]:
x = data['Text']
y = data['Category']

In [18]:
X, test_x, y, test_y = train_test_split(x, y, test_size=0.10, random_state=0, shuffle=True)
tf = TfidfVectorizer()
tf.fit(X)
len(tf.get_feature_names())


7337

In [155]:
print(len(test_x))
print(len(x))

283
2826


In [19]:
X = tf.transform(X)
test_X = tf.transform(test_x)


In [20]:
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score

In [53]:
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 10, 12],
    'random_state': [13]
}
kf = KFold(n_splits=4, random_state=42, shuffle=False)

In [44]:
imba_pipeline = make_pipeline(SMOTE(random_state=42), 
                              RandomForestClassifier(n_estimators=100, random_state=13))
cross_val_score(imba_pipeline, X, y,  cv=kf).mean()

0.7015339474075174

In [63]:
# new_params = {'LogisticRegression__' + key: params[key] for key in params}
new_params = {'randomforestclassifier__' + key: params[key] for key in params}
grid_imba = GridSearchCV(imba_pipeline, param_grid=new_params, cv=kf, scoring='recall_weighted',
                        return_train_score=True)

In [64]:
grid_imba.fit(X,y)

GridSearchCV(cv=KFold(n_splits=4, random_state=42, shuffle=False),
             estimator=Pipeline(steps=[('smote', SMOTE(random_state=42)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=13))]),
             param_grid={'randomforestclassifier__max_depth': [4, 6, 10, 12],
                         'randomforestclassifier__n_estimators': [50, 100, 200],
                         'randomforestclassifier__random_state': [13]},
             return_train_score=True, scoring='recall_weighted')

In [65]:
grid_imba.best_params_

{'randomforestclassifier__max_depth': 12,
 'randomforestclassifier__n_estimators': 200,
 'randomforestclassifier__random_state': 13}

In [66]:
grid_imba.best_score_

0.6044012776704799

# Train/Split/Validate


In [156]:
cats = ['Aanrijding',
'Agressie en extreme lichamelijke inspanning',
'Andere vormen van contact met voorwerpen',
'BHV inzet op locatie',
'Gevaarlijke stoffen, brand en explosies',
'Machines en handgereedschap',
'Vallen (van personen of voorwerpen)']


In [157]:
x = data["Text"]
y = data["Category"]
X, test_x, y, test_y = train_test_split(x, y, test_size=0.1, random_state=0, shuffle=True)

X, val_x, y, val_y = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True)

In [158]:
tf = TfidfVectorizer()
tf.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [159]:
X = tf.transform(X)
test_X = tf.transform(test_x)
val_x = tf.transform(val_x)

In [160]:
cv = y.value_counts()[0]

In [ ]:
print (cv)

# Baseline

In [236]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
dummy_score = dummy_clf.score(val_x, val_y)
print(dummy_score)

0.5725490196078431


In [162]:
from sklearn.svm import LinearSVC
SVM_cl= LinearSVC(penalty="l1", loss='squared_hinge',dual=False )
SVM_cl.fit(X,y)


LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
          verbose=0)

In [164]:
SVM_cl.score(val_x, val_y)

0.7568627450980392

In [167]:
preds = SVM_cl.predict(val_x)

In [168]:
print("Recall:" + recall_score(preds, val_y, average='macro').astype(str))
print("Precision:" + precision_score(preds, val_y, average='macro').astype(str))
print("F1:" + f1_score(preds, val_y, average='macro').astype(str))

Recall:0.7511512965901951
Precision:0.56563094970231
F1:0.6326530612244898


In [169]:
pd.DataFrame(confusion_matrix(preds, val_y, labels=y.unique()), columns =y.unique(), index=y.unique())


,Vallen (van personen of voorwerpen),Machines en handgereedschap,Aanrijding,"Gevaarlijke stoffen, brand en explosies",BHV inzet op locatie,Andere vormen van contact met voorwerpen,Agressie en extreme lichamelijke inspanning
Vallen (van personen of voorwerpen),133,9,4,2,2,15,1
Machines en handgereedschap,5,20,1,1,1,5,0
Aanrijding,1,0,8,0,0,0,0
"Gevaarlijke stoffen, brand en explosies",0,0,0,1,1,1,0
BHV inzet op locatie,0,0,0,0,8,0,0
Andere vormen van contact met voorwerpen,7,4,0,1,1,22,0
Agressie en extreme lichamelijke inspanning,0,0,0,0,0,0,1


In [187]:
print("Train score:", SVM_cl.score(val_x, val_y))
print("Test score:",SVM_cl.score(test_X, test_y))

Train score: 0.7568627450980392
Test score: 0.6607773851590106


In [191]:
lr_gc = LogisticRegression(random_state=0, penalty='l2', C=10 )
lr_gc.fit(X,y)

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [192]:
lr_gc.score(val_x, val_y)

0.7450980392156863

In [193]:
preds = lr_gc.predict(val_x)
print("Recall:" + recall_score(preds, val_y, average='macro').astype(str))
print("Precision:" + precision_score(preds, val_y, average='macro').astype(str))
print("F1:" + f1_score(preds, val_y, average='macro').astype(str))

Recall:0.702295918367347
Precision:0.4655232948001345
F1:0.5253103686531666


C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [237]:
print("Val score:", lr_gc.score(val_x, val_y))
print("Test score:",lr_gc.score(test_X, test_y))

Val score: 0.7450980392156863
Test score: 0.657243816254417


In [213]:
from sklearn.naive_bayes import MultinomialNB
NB_cl = MultinomialNB(alpha=1.0) 
NB_cl.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [214]:
MultinomialNB.get_params(MultinomialNB).keys()

C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


dict_keys(['alpha', 'class_prior', 'fit_prior'])

In [215]:
NB_cl.score(val_x, val_y)

0.6352941176470588

In [216]:
preds = NB_cl.predict(val_x)
print("Recall:" + recall_score(preds, val_y, average='macro').astype(str))
print("Precision:" + precision_score(preds, val_y, average='macro').astype(str))
print("F1:" + f1_score(preds, val_y, average='macro').astype(str))

Recall:0.4737191078963231
Precision:0.2280742513300653
F1:0.24609620094318618


C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [218]:
print("Val score:", NB_cl.score(val_x, val_y))
print("Test score:",NB_cl.score(test_X, test_y))

Val score: 0.6352941176470588
Test score: 0.5159010600706714


In [229]:
from sklearn.tree import DecisionTreeClassifier
DT_cl = DecisionTreeClassifier( criterion="entropy")
DT_cl.fit(X,y)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [230]:
DT_cl.score(val_x, val_y)
preds = DT_cl.predict(val_x)
print("Recall:" + recall_score(preds, val_y, average='macro').astype(str))
print("Precision:" + precision_score(preds, val_y, average='macro').astype(str))
print("F1:" + f1_score(preds, val_y, average='macro').astype(str))

Recall:0.44963386045928166
Precision:0.48231051098206246
F1:0.4548934862569182


In [231]:
print("Val score:", DT_cl.score(val_x, val_y))
print("Test score:",DT_cl.score(test_X, test_y))

Val score: 0.6274509803921569
Test score: 0.568904593639576


# GridSearch

In [241]:
model_params= {"svm": {"model":svm.SVC(gamma="auto"), "params": {"C": [1,10,20], "kernel":["rbf", "linear"], "penalty":["l1", "L2"], "dual": ["False", "True"], "loss":["squared_hinge"], "class_weight":[0.05, 0.95, 1, 20],  cv=cv}}, 
              "decision_tree": {"model":DecisionTreeClassifier(),"params": {"n_estimators": [1,10], "criterion":['gini','entropy'], "class_weight":[0.05, 0.95, 1, 20],  cv=cv}}, 
               "naive_bayes":{"model":MultinomialNB(),"params": {"C": [1,10],"class_weight":[0.05, 0.95, 1, 20] }}}
               #"LR_gs":{"model":LogisticRegression(random_state=0),"params":{'penalty':['l1', 'l2'], 'C':[0.1, 1, 5, 10]}, scoring='f1_macro', n_jobs=-1, verbose=3, cv=cv }}
               

SyntaxError: invalid syntax (<ipython-input-241-c2499326f3f2>, line 1)

In [253]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.svm import SVC
LR_gs = GridSearchCV(LogisticRegression(random_state=0), {'penalty':['l1', 'l2'], 'C':[0.1, 1, 5, 10]}, scoring='f1_macro', n_jobs=-1, verbose=3, cv=cv)
SVM_gs= GridSearchCV(LinearSVC(),  {"C": [10], "kernel":["rbf", "linear"], "penalty":["l1", "L2"], "loss":["squared_hinge"], "class_weight":[0.05]},  cv=cv)

In [ ]:
LR_gs.fit(X, y)
LR_gs.best_estimator_

Fitting 1202 folds for each of 8 candidates, totalling 9616 fits


C:\Users\byri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 57 members, which is less than n_splits=1202.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1883 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 2043 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 2267 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 4195 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 4547 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 7885 tasks      | elapsed: 32.2min


In [ ]:
SVM_gs.fit(X, y)
SVM_gs.best_estimator_